In [ ]:
# i wonder if we should look at how the scores are distributed within EACH of Humplocs classes? 
# wtf is going on with this: SSQSTIPIVGIVAGL?? 
# mask is NOT WORKING RIGHT, pretty sure of that, that needs attention 

# added preprocessing

# todo: we need to find a way to do an inner join on humploc and HPA, but for that, we need to somehow look up for HPA the corresponding Uniprot identifiers. I have looked into it: i think we can map the ensembl ids from HPA using uniprot search features (we want a) only reviewed entries and b) for HPA only human entries), like so: http://www.uniprot.org/uniprot/?query=ENSG00000003402+AND+reviewed%3Ayes+AND+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22&sort=score 

# this seems to give 1 result per query. however, I currently do not know how to automate it, i.e. how to get the query result as something easily parseable or such. Once that is possible, we can just put the result into an additional column and then use the same logic that was used for the other comparison. This is necessary to compare HUMPLOC to HPA.

In [120]:
import re
import pandas as pd

In [143]:
data = None
# top = ['AC\n', 'ID\t\n', 'SEQ\n', 'Predicted Subcellular location by Hum-mPLoc 3.0\t\n', 'Outputted prediction scores for each location from Hum-mPLoc 3.0: Centrosome,Cytoplasm,Cytoskeleton,Endoplasmic reticulum,Endosome,Extracellular,Golgi apparatus,Lysosome,Mitochondrion,Nucleus,Peroxisome,Plasma membrane\n', '//\n']

with open('humploc_predictions.txt', 'r') as original: 
    data = original.readlines()

# data = top + data[19:-1]
data = data[19:-1]
textdata = ''.join(data)
replaceddata = re.sub('\n{2,}','\n\n', textdata) 

with open('humploc_predictions.tsv', 'w') as woooowiamoutofnames:
    woooowiamoutofnames.write(replaceddata)

In [144]:
# humploc_predictions_txt = "humploc_predictions.txt"
# in uniprot file, "subcellular location" can be empty

linecounter = 0
secondcounter = 0 
counter = 0

h_ac = None
h_id = None
h_seq = "" 
h_loc = None 
h_scores = None

subcellular_location_tsv = "subcellular_location.tsv"
humploc_predictions_txt = "humploc_predictions.tsv" # tsv better than txt (because I removed the header info manually; this should be in the code instead) Also, tsv has a correction (two newlines removed into one)
uniprot_results_tsv = "uniprot_results.tsv"
output_filename = "comparison_output.csv"

begin_flag = True
seq_flag = False
dic = {}

df = pd.DataFrame(columns=['AC', 'ID', 'SEQ', 'PRED_LOC', 'SCORES'])

with open(humploc_predictions_txt) as humploc_text:
    for line in humploc_text: 
        counter += 1
        linecounter += 1
        if not begin_flag:
            secondcounter += 1
        if linecounter == 1:
            h_ac = line.strip()
        if linecounter == 2:
            h_id = line.strip()
        if linecounter == 3:
            seq_flag = True
        if not line.strip():
            seq_flag = False
            begin_flag = False 
        if seq_flag:
            h_seq = h_seq + line.strip()
        if secondcounter == 1:
            h_loc = line.strip()
        if secondcounter == 2: 
            h_scores = line.strip()
        if re.search('//', line):
            linecounter = 0
            secondcounter = 0
            begin_flag = True
            dic['AC'] = h_ac
            dic['ID'] = h_id
            dic['SEQ'] = h_seq
            dic['PRED_LOC'] = h_loc
            dic['SCORES'] = h_scores
            h_ac = None
            h_id = None
            h_seq = "" 
            h_loc = None 
            h_scores = None
            df = df.append(dic, ignore_index=True)
            dic = {}
print('created a Dataframe from Humploc data')

created a Dataframe from Humploc data


In [145]:
humploc_df = df.fillna('') # subtitute NaNs with ''
humploc_df

,AC,ID,SEQ,PRED_LOC,SCORES
0,P62258,1433E_HUMAN,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,Cytoplasm.,"-1.158095,1.053253,-1.496686,-2.250498,-1.8343..."
1,Q04917,1433F_HUMAN,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,Cytoplasm.,"-1.254769,0.487241,-1.091541,-2.543192,-2.0037..."
2,P61981,1433G_HUMAN,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,Cytoplasm.,"-1.222702,0.689057,-1.099006,-2.577572,-2.0572..."
3,P31947,1433S_HUMAN,MERASLIQKAKLAEQAERYEDMAAFMKGAVEKGEELSCEERNLLSV...,Cytoplasm.,"-1.241884,1.438412,-1.192020,-2.279416,-1.7292..."
4,P27348,1433T_HUMAN,MEKTELIQKAKLAEQAERYDDMATCMKAVTEQGAELSNEERNLLSV...,Cytoplasm.,"-1.173382,2.196761,-1.051799,-1.719705,-1.7929..."
5,P63104,1433Z_HUMAN,MDKNELVQKAKLAEQAERYDDMAACMKSVTEQGAELSNEERNLLSV...,Cytoplasm.,"-1.169426,2.205039,-1.041187,-1.750936,-1.8200..."
6,P30443,1A01_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Golgi apparatus.,"-1.825552,-4.784307,-2.787740,0.483626,-0.0560..."
7,P01892,1A02_HUMAN,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Endoplasmic reticulum. Golgi appar...,"-1.778965,-4.829878,-2.702989,0.544520,0.11073..."
8,P04439,1A03_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Endoplasmic reticulum. Golgi appar...,"-1.845178,-4.747686,-2.862900,0.513883,0.01748..."
9,P13746,1A11_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFYTSVSRPGRGEPRF...,Endosome. Golgi apparatus.,"-1.828251,-4.713617,-2.850621,0.404953,-0.0494..."


In [157]:
humploc_df = humploc_df.rename(columns={'SCORES': 'Centrosome,Cytoplasm,Cytoskeleton,Endoplasmic reticulum,Endosome,Extracellular,Golgi apparatus,Lysosome,Mitochondrion,Nucleus,Peroxisome,Plasma membrane'})

humploc_df[["Centrosome", "Cytoplasm", "Cytoskeleton", "Endoplasmic reticulum", "Endosome", "Extracellular", "Golgi apparatus", "Lysosome", "Mitochondrion", "Nucleus", "Peroxisome", "Plasma membrane"]] = humploc_df['Centrosome,Cytoplasm,Cytoskeleton,Endoplasmic reticulum,Endosome,Extracellular,Golgi apparatus,Lysosome,Mitochondrion,Nucleus,Peroxisome,Plasma membrane'].str.split(',', expand=True)

humploc_df

,AC,ID,SEQ,PRED_LOC,"Centrosome,Cytoplasm,Cytoskeleton,Endoplasmic reticulum,Endosome,Extracellular,Golgi apparatus,Lysosome,Mitochondrion,Nucleus,Peroxisome,Plasma membrane",Centrosome,Cytoplasm,Cytoskeleton,Endoplasmic reticulum,Endosome,Extracellular,Golgi apparatus,Lysosome,Mitochondrion,Nucleus,Peroxisome,Plasma membrane
0,P62258,1433E_HUMAN,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,Cytoplasm.,"-1.158095,1.053253,-1.496686,-2.250498,-1.8343...",-1.158095,1.053253,-1.496686,-2.250498,-1.834363,-3.033166,-1.462610,-2.451649,-1.511337,-0.701560,-1.574325,-1.554137
1,Q04917,1433F_HUMAN,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,Cytoplasm.,"-1.254769,0.487241,-1.091541,-2.543192,-2.0037...",-1.254769,0.487241,-1.091541,-2.543192,-2.003732,-1.992132,-1.921637,-3.027127,-1.496806,-0.586117,-1.808539,-0.623766
2,P61981,1433G_HUMAN,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,Cytoplasm.,"-1.222702,0.689057,-1.099006,-2.577572,-2.0572...",-1.222702,0.689057,-1.099006,-2.577572,-2.057291,-1.817514,-1.891795,-3.266725,-1.536691,-0.622111,-1.828860,-0.667136
3,P31947,1433S_HUMAN,MERASLIQKAKLAEQAERYEDMAAFMKGAVEKGEELSCEERNLLSV...,Cytoplasm.,"-1.241884,1.438412,-1.192020,-2.279416,-1.7292...",-1.241884,1.438412,-1.192020,-2.279416,-1.729267,-3.304249,-2.032787,-2.130410,-2.091538,-0.542656,-1.589618,-1.227719
4,P27348,1433T_HUMAN,MEKTELIQKAKLAEQAERYDDMATCMKAVTEQGAELSNEERNLLSV...,Cytoplasm.,"-1.173382,2.196761,-1.051799,-1.719705,-1.7929...",-1.173382,2.196761,-1.051799,-1.719705,-1.792961,-2.679922,-1.175184,-1.926964,-2.116135,-1.159717,-1.820097,-1.086497
5,P63104,1433Z_HUMAN,MDKNELVQKAKLAEQAERYDDMAACMKSVTEQGAELSNEERNLLSV...,Cytoplasm.,"-1.169426,2.205039,-1.041187,-1.750936,-1.8200...",-1.169426,2.205039,-1.041187,-1.750936,-1.820066,-2.886048,-1.174116,-1.952768,-2.098092,-1.155437,-1.826804,-1.100602
6,P30443,1A01_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Golgi apparatus.,"-1.825552,-4.784307,-2.787740,0.483626,-0.0560...",-1.825552,-4.784307,-2.787740,0.483626,-0.056057,-1.773511,1.731748,-0.382207,-2.701896,-3.146799,-1.622583,-0.346744
7,P01892,1A02_HUMAN,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Endoplasmic reticulum. Golgi appar...,"-1.778965,-4.829878,-2.702989,0.544520,0.11073...",-1.778965,-4.829878,-2.702989,0.544520,0.110736,-1.653822,2.127933,-0.730438,-2.901703,-3.232460,-1.728008,-0.508353
8,P04439,1A03_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Endoplasmic reticulum. Golgi appar...,"-1.845178,-4.747686,-2.862900,0.513883,0.01748...",-1.845178,-4.747686,-2.862900,0.513883,0.017483,-1.774493,1.650542,-0.320197,-2.787355,-3.112934,-1.605647,-0.440056
9,P13746,1A11_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFYTSVSRPGRGEPRF...,Endosome. Golgi apparatus.,"-1.828251,-4.713617,-2.850621,0.404953,-0.0494...",-1.828251,-4.713617,-2.850621,0.404953,-0.049402,-1.747025,1.679218,-0.452545,-2.624218,-3.110929,-1.659584,-0.348176


In [159]:
humploc_df[["Centrosome", "Cytoplasm", "Cytoskeleton", "Endoplasmic reticulum", "Endosome", "Extracellular", "Golgi apparatus", "Lysosome", "Mitochondrion", "Nucleus", "Peroxisome", "Plasma membrane"]] = humploc_df[["Centrosome", "Cytoplasm", "Cytoskeleton", "Endoplasmic reticulum", "Endosome", "Extracellular", "Golgi apparatus", "Lysosome", "Mitochondrion", "Nucleus", "Peroxisome", "Plasma membrane"]].astype(float)

In [162]:
humploc_df["HighScore"] = humploc_df[["Centrosome", "Cytoplasm", "Cytoskeleton", "Endoplasmic reticulum", "Endosome", "Extracellular", "Golgi apparatus", "Lysosome", "Mitochondrion", "Nucleus", "Peroxisome", "Plasma membrane"]].max(axis=1)

In [166]:
humploc_df.drop(humploc_df[humploc_df["HighScore"] < 0.0].index) # this can now be used later, like so: humploc_df = humploc_df.drop(humploc_df[humploc_df["HighScore"] < 0.0].index) # also remember, we're DROPPING lines that are <0.0 

,AC,ID,SEQ,PRED_LOC,"Centrosome,Cytoplasm,Cytoskeleton,Endoplasmic reticulum,Endosome,Extracellular,Golgi apparatus,Lysosome,Mitochondrion,Nucleus,Peroxisome,Plasma membrane",Centrosome,Cytoplasm,Cytoskeleton,Endoplasmic reticulum,Endosome,Extracellular,Golgi apparatus,Lysosome,Mitochondrion,Nucleus,Peroxisome,Plasma membrane,HighScore
0,P62258,1433E_HUMAN,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,Cytoplasm.,"-1.158095,1.053253,-1.496686,-2.250498,-1.8343...",-1.158095,1.053253,-1.496686,-2.250498,-1.834363,-3.033166,-1.462610,-2.451649,-1.511337,-0.701560,-1.574325,-1.554137,1.053253
1,Q04917,1433F_HUMAN,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,Cytoplasm.,"-1.254769,0.487241,-1.091541,-2.543192,-2.0037...",-1.254769,0.487241,-1.091541,-2.543192,-2.003732,-1.992132,-1.921637,-3.027127,-1.496806,-0.586117,-1.808539,-0.623766,0.487241
2,P61981,1433G_HUMAN,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,Cytoplasm.,"-1.222702,0.689057,-1.099006,-2.577572,-2.0572...",-1.222702,0.689057,-1.099006,-2.577572,-2.057291,-1.817514,-1.891795,-3.266725,-1.536691,-0.622111,-1.828860,-0.667136,0.689057
3,P31947,1433S_HUMAN,MERASLIQKAKLAEQAERYEDMAAFMKGAVEKGEELSCEERNLLSV...,Cytoplasm.,"-1.241884,1.438412,-1.192020,-2.279416,-1.7292...",-1.241884,1.438412,-1.192020,-2.279416,-1.729267,-3.304249,-2.032787,-2.130410,-2.091538,-0.542656,-1.589618,-1.227719,1.438412
4,P27348,1433T_HUMAN,MEKTELIQKAKLAEQAERYDDMATCMKAVTEQGAELSNEERNLLSV...,Cytoplasm.,"-1.173382,2.196761,-1.051799,-1.719705,-1.7929...",-1.173382,2.196761,-1.051799,-1.719705,-1.792961,-2.679922,-1.175184,-1.926964,-2.116135,-1.159717,-1.820097,-1.086497,2.196761
5,P63104,1433Z_HUMAN,MDKNELVQKAKLAEQAERYDDMAACMKSVTEQGAELSNEERNLLSV...,Cytoplasm.,"-1.169426,2.205039,-1.041187,-1.750936,-1.8200...",-1.169426,2.205039,-1.041187,-1.750936,-1.820066,-2.886048,-1.174116,-1.952768,-2.098092,-1.155437,-1.826804,-1.100602,2.205039
6,P30443,1A01_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Golgi apparatus.,"-1.825552,-4.784307,-2.787740,0.483626,-0.0560...",-1.825552,-4.784307,-2.787740,0.483626,-0.056057,-1.773511,1.731748,-0.382207,-2.701896,-3.146799,-1.622583,-0.346744,1.731748
7,P01892,1A02_HUMAN,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Endoplasmic reticulum. Golgi appar...,"-1.778965,-4.829878,-2.702989,0.544520,0.11073...",-1.778965,-4.829878,-2.702989,0.544520,0.110736,-1.653822,2.127933,-0.730438,-2.901703,-3.232460,-1.728008,-0.508353,2.127933
8,P04439,1A03_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Endoplasmic reticulum. Golgi appar...,"-1.845178,-4.747686,-2.862900,0.513883,0.01748...",-1.845178,-4.747686,-2.862900,0.513883,0.017483,-1.774493,1.650542,-0.320197,-2.787355,-3.112934,-1.605647,-0.440056,1.650542
9,P13746,1A11_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFYTSVSRPGRGEPRF...,Endosome. Golgi apparatus.,"-1.828251,-4.713617,-2.850621,0.404953,-0.0494...",-1.828251,-4.713617,-2.850621,0.404953,-0.049402,-1.747025,1.679218,-0.452545,-2.624218,-3.110929,-1.659584,-0.348176,1.679218


In [158]:
# humploc_df.to_csv(path_or_buf="removeme.csv", sep='\t') # for exporting

In [152]:
hpa = pd.read_csv(subcellular_location_tsv, sep="\t")
hpa = hpa.fillna('')
hpa

,Gene,Gene name,Reliability,Validated,Supported,Approved,Uncertain,Single-cell variation intensity,Single-cell variation spatial,Cell cycle dependency,GO id
0,ENSG00000000003,TSPAN6,Approved,,,Cytosol,,,,,Cytosol (GO:0005829)
1,ENSG00000000457,SCYL3,Uncertain,,,,Microtubules;Nuclear bodies,,,,Microtubules (GO:0015630);Nuclear bodies (GO:0...
2,ENSG00000000460,C1orf112,Approved,,,Mitochondria,,,,,Mitochondria (GO:0005739)
3,ENSG00000000938,FGR,Approved,,,Aggresome;Plasma membrane,,,,,Aggresome (GO:0016235);Plasma membrane (GO:000...
4,ENSG00000000971,CFH,Approved,,,Vesicles,,,,,Vesicles (GO:0043231)
5,ENSG00000001084,GCLC,Approved,,,Cytosol;Nucleoli;Nucleus,,,,,Cytosol (GO:0005829);Nucleoli (GO:0005730);Nuc...
6,ENSG00000001167,NFYA,Supported,,Nucleoplasm,,,,,,Nucleoplasm (GO:0005654)
7,ENSG00000001460,STPG1,Approved,,,Nucleoplasm,,,,,Nucleoplasm (GO:0005654)
8,ENSG00000001461,NIPAL3,Approved,,,Nucleoplasm,,,,,Nucleoplasm (GO:0005654)
9,ENSG00000001497,LAS1L,Approved,,Nucleoplasm,Cytosol;Microtubule organizing center,,,,,Cytosol (GO:0005829);Microtubule organizing ce...


In [151]:
uniprot = pd.read_csv(uniprot_results_tsv, sep="\t")
uniprot = uniprot.fillna('')
uniprot

,Entry,Entry name,Status,Protein names,Gene names,Subcellular location [CC],Gene ontology (GO)
0,P04637,P53_HUMAN,reviewed,Cellular tumor antigen p53 (Antigen NY-CO-13) ...,TP53 P53,SUBCELLULAR LOCATION: Cytoplasm. Nucleus. Nucl...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...
1,Q14524,SCN5A_HUMAN,reviewed,Sodium channel protein type 5 subunit alpha (H...,SCN5A,SUBCELLULAR LOCATION: Cell membrane {ECO:00002...,caveola [GO:0005901]; cell surface [GO:0009986...
2,P05067,A4_HUMAN,reviewed,Amyloid-beta A4 protein (ABPP) (APPI) (APP) (A...,APP A4 AD1,SUBCELLULAR LOCATION: Membrane; Single-pass ty...,amyloid-beta complex [GO:0106003]; apical part...
3,P00533,EGFR_HUMAN,reviewed,Epidermal growth factor receptor (EC 2.7.10.1)...,EGFR ERBB ERBB1 HER1,SUBCELLULAR LOCATION: Cell membrane; Single-pa...,apical plasma membrane [GO:0016324]; basolater...
4,P35555,FBN1_HUMAN,reviewed,Fibrillin-1 [Cleaved into: Asprosin],FBN1 FBN,SUBCELLULAR LOCATION: Secreted {ECO:0000269|Pu...,basement membrane [GO:0005604]; endoplasmic re...
5,P35222,CTNB1_HUMAN,reviewed,Catenin beta-1 (Beta-catenin),CTNNB1 CTNNB OK/SW-cl.35 PRO2286,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|P...,adherens junction [GO:0005912]; apical part of...
6,P10275,ANDR_HUMAN,reviewed,Androgen receptor (Dihydrotestosterone recepto...,AR DHTR NR3C4,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|Pub...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...
7,P00451,FA8_HUMAN,reviewed,Coagulation factor VIII (Antihemophilic factor...,F8 F8C,"SUBCELLULAR LOCATION: Secreted, extracellular ...",COPII-coated ER to Golgi transport vesicle [GO...
8,Q5S007,LRRK2_HUMAN,reviewed,Leucine-rich repeat serine/threonine-protein k...,LRRK2 PARK8,SUBCELLULAR LOCATION: Membrane; Peripheral mem...,amphisome [GO:0044753]; autolysosome [GO:00447...
9,Q8WZ42,TITIN_HUMAN,reviewed,Titin (EC 2.7.11.1) (Connectin) (Rhabdomyosarc...,TTN,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000305|P...,condensed nuclear chromosome [GO:0000794]; cyt...


In [153]:
# inner join on the AC identifier using humploc and uniprot results
proteins_in_uniprot_and_humploc_using_AC = pd.merge(uniprot, humploc_df, left_on="Entry", right_on="AC", how="inner")

In [154]:
df = proteins_in_uniprot_and_humploc_using_AC
df

,Entry,Entry name,Status,Protein names,Gene names,Subcellular location [CC],Gene ontology (GO),AC,ID,SEQ,PRED_LOC,SCORES
0,P04637,P53_HUMAN,reviewed,Cellular tumor antigen p53 (Antigen NY-CO-13) ...,TP53 P53,SUBCELLULAR LOCATION: Cytoplasm. Nucleus. Nucl...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,P04637,P53_HUMAN,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,Nucleus.,"-1.421297,-0.829262,-1.701711,-2.569025,-1.864..."
1,Q14524,SCN5A_HUMAN,reviewed,Sodium channel protein type 5 subunit alpha (H...,SCN5A,SUBCELLULAR LOCATION: Cell membrane {ECO:00002...,caveola [GO:0005901]; cell surface [GO:0009986...,Q14524,SCN5A_HUMAN,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...,Plasma membrane.,"-1.826585,-1.933985,-1.308556,-2.743710,-2.408..."
2,P05067,A4_HUMAN,reviewed,Amyloid-beta A4 protein (ABPP) (APPI) (APP) (A...,APP A4 AD1,SUBCELLULAR LOCATION: Membrane; Single-pass ty...,amyloid-beta complex [GO:0106003]; apical part...,P05067,A4_HUMAN,MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...,Extracellular.,"-1.675579,-1.089203,-1.628664,-1.264180,-0.822..."
3,P00533,EGFR_HUMAN,reviewed,Epidermal growth factor receptor (EC 2.7.10.1)...,EGFR ERBB ERBB1 HER1,SUBCELLULAR LOCATION: Cell membrane; Single-pa...,apical plasma membrane [GO:0016324]; basolater...,P00533,EGFR_HUMAN,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,Plasma membrane.,"-1.241858,-2.066043,-1.782397,-0.825578,-1.115..."
4,P35555,FBN1_HUMAN,reviewed,Fibrillin-1 [Cleaved into: Asprosin],FBN1 FBN,SUBCELLULAR LOCATION: Secreted {ECO:0000269|Pu...,basement membrane [GO:0005604]; endoplasmic re...,P35555,FBN1_HUMAN,MRRGRLLEIALGFTVLLASYTSHGADANLEAGNVKETRASRAKRRG...,Plasma membrane.,"-1.593076,-1.352582,-2.480642,-2.095503,-2.023..."
5,P35222,CTNB1_HUMAN,reviewed,Catenin beta-1 (Beta-catenin),CTNNB1 CTNNB OK/SW-cl.35 PRO2286,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|P...,adherens junction [GO:0005912]; apical part of...,P35222,CTNB1_HUMAN,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,Cytoplasm.,"-0.909892,2.289614,-0.170275,-2.687533,-2.5651..."
6,P10275,ANDR_HUMAN,reviewed,Androgen receptor (Dihydrotestosterone recepto...,AR DHTR NR3C4,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|Pub...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,P10275,ANDR_HUMAN,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,Nucleus.,"-1.609735,-0.518090,-2.641733,-2.268793,-1.834..."
7,P00451,FA8_HUMAN,reviewed,Coagulation factor VIII (Antihemophilic factor...,F8 F8C,"SUBCELLULAR LOCATION: Secreted, extracellular ...",COPII-coated ER to Golgi transport vesicle [GO...,P00451,FA8_HUMAN,MQIELSTCFFLCLLRFCFSATRRYYLGAVELSWDYMQSDLGELPVD...,Endoplasmic reticulum. Golgi apparatus.,"-1.286432,-2.423427,-1.796176,-1.866689,0.5352..."
8,Q5S007,LRRK2_HUMAN,reviewed,Leucine-rich repeat serine/threonine-protein k...,LRRK2 PARK8,SUBCELLULAR LOCATION: Membrane; Peripheral mem...,amphisome [GO:0044753]; autolysosome [GO:00447...,Q5S007,LRRK2_HUMAN,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Cytoplasm.,"-1.543625,1.100506,-1.454457,-1.534116,-1.0217..."
9,Q8WZ42,TITIN_HUMAN,reviewed,Titin (EC 2.7.11.1) (Connectin) (Rhabdomyosarc...,TTN,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000305|P...,condensed nuclear chromosome [GO:0000794]; cyt...,Q8WZ42,TITIN_HUMAN,MTTQAPTFTQPLQSVVVLEGSTATFEAHISGFPVPEVSWFRDGQVI...,Cytoplasm. Cytoskeleton. Plasma membrane.,"-1.418561,2.064323,0.189796,-1.398633,-2.24081..."


In [155]:
mask = pd.DataFrame({
           'UNIPROTS': (df["Subcellular location [CC]"] + df["Gene ontology (GO)"]).str.lower().map(set),
           'HUMPLOC': df["PRED_LOC"].str.lower().map(set)
       }).apply(lambda row: bool(row['UNIPROTS'].intersection(row['HUMPLOC'])), axis=1)

df[mask].reset_index(drop=True)

,Entry,Entry name,Status,Protein names,Gene names,Subcellular location [CC],Gene ontology (GO),AC,ID,SEQ,PRED_LOC,SCORES
0,P04637,P53_HUMAN,reviewed,Cellular tumor antigen p53 (Antigen NY-CO-13) ...,TP53 P53,SUBCELLULAR LOCATION: Cytoplasm. Nucleus. Nucl...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,P04637,P53_HUMAN,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,Nucleus.,"-1.421297,-0.829262,-1.701711,-2.569025,-1.864..."
1,Q14524,SCN5A_HUMAN,reviewed,Sodium channel protein type 5 subunit alpha (H...,SCN5A,SUBCELLULAR LOCATION: Cell membrane {ECO:00002...,caveola [GO:0005901]; cell surface [GO:0009986...,Q14524,SCN5A_HUMAN,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...,Plasma membrane.,"-1.826585,-1.933985,-1.308556,-2.743710,-2.408..."
2,P05067,A4_HUMAN,reviewed,Amyloid-beta A4 protein (ABPP) (APPI) (APP) (A...,APP A4 AD1,SUBCELLULAR LOCATION: Membrane; Single-pass ty...,amyloid-beta complex [GO:0106003]; apical part...,P05067,A4_HUMAN,MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...,Extracellular.,"-1.675579,-1.089203,-1.628664,-1.264180,-0.822..."
3,P00533,EGFR_HUMAN,reviewed,Epidermal growth factor receptor (EC 2.7.10.1)...,EGFR ERBB ERBB1 HER1,SUBCELLULAR LOCATION: Cell membrane; Single-pa...,apical plasma membrane [GO:0016324]; basolater...,P00533,EGFR_HUMAN,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,Plasma membrane.,"-1.241858,-2.066043,-1.782397,-0.825578,-1.115..."
4,P35555,FBN1_HUMAN,reviewed,Fibrillin-1 [Cleaved into: Asprosin],FBN1 FBN,SUBCELLULAR LOCATION: Secreted {ECO:0000269|Pu...,basement membrane [GO:0005604]; endoplasmic re...,P35555,FBN1_HUMAN,MRRGRLLEIALGFTVLLASYTSHGADANLEAGNVKETRASRAKRRG...,Plasma membrane.,"-1.593076,-1.352582,-2.480642,-2.095503,-2.023..."
5,P35222,CTNB1_HUMAN,reviewed,Catenin beta-1 (Beta-catenin),CTNNB1 CTNNB OK/SW-cl.35 PRO2286,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|P...,adherens junction [GO:0005912]; apical part of...,P35222,CTNB1_HUMAN,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,Cytoplasm.,"-0.909892,2.289614,-0.170275,-2.687533,-2.5651..."
6,P10275,ANDR_HUMAN,reviewed,Androgen receptor (Dihydrotestosterone recepto...,AR DHTR NR3C4,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|Pub...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,P10275,ANDR_HUMAN,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,Nucleus.,"-1.609735,-0.518090,-2.641733,-2.268793,-1.834..."
7,P00451,FA8_HUMAN,reviewed,Coagulation factor VIII (Antihemophilic factor...,F8 F8C,"SUBCELLULAR LOCATION: Secreted, extracellular ...",COPII-coated ER to Golgi transport vesicle [GO...,P00451,FA8_HUMAN,MQIELSTCFFLCLLRFCFSATRRYYLGAVELSWDYMQSDLGELPVD...,Endoplasmic reticulum. Golgi apparatus.,"-1.286432,-2.423427,-1.796176,-1.866689,0.5352..."
8,Q5S007,LRRK2_HUMAN,reviewed,Leucine-rich repeat serine/threonine-protein k...,LRRK2 PARK8,SUBCELLULAR LOCATION: Membrane; Peripheral mem...,amphisome [GO:0044753]; autolysosome [GO:00447...,Q5S007,LRRK2_HUMAN,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Cytoplasm.,"-1.543625,1.100506,-1.454457,-1.534116,-1.0217..."
9,Q8WZ42,TITIN_HUMAN,reviewed,Titin (EC 2.7.11.1) (Connectin) (Rhabdomyosarc...,TTN,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000305|P...,condensed nuclear chromosome [GO:0000794]; cyt...,Q8WZ42,TITIN_HUMAN,MTTQAPTFTQPLQSVVVLEGSTATFEAHISGFPVPEVSWFRDGQVI...,Cytoplasm. Cytoskeleton. Plasma membrane.,"-1.418561,2.064323,0.189796,-1.398633,-2.24081..."


In [ ]:
# testnumbs = "-1.158095,1.053253,-1.496686,-2.250498,-1.834363,-3.033166,-1.462610,-2.451649,-1.511337,-0.701560,-1.574325,-1.554137"
# [type(x) for x in testnumbs.split(',')]
# max([float(x) for x in testnumbs.split(',')])
# humploc_locs_list = ["Centrosome","Cytoplasm","Cytoskeleton","Endoplasmic reticulum","Endosome","Extracellular","Golgi apparatus","Lysosome","Mitochondrion","Nucleus","Peroxisome","Plasma membrane"]
# dic = dict(zip(humploc_locs_list, [float(x) for x in testnumbs.split(',')]))
# dic
# max(dic, key=dic.get)  # https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary 
# humploc_df[['Centrosome']] = humploc_df[['Centrosome']].astype(float)

In [5]:
# this block is probably unnecessary, have it here for reference

# def main(input_filename, output_filename):
#     with open(output_filename, 'w') as output_file:
#         write_stats(stats, output_file)
#         
# def analyze_humploc(f):
#     humploc_loc_re = re.compile(r'Centrosome\b | \bCytoplasm\b | \bCytoskeleton\b | \bEndoplasmic reticulum\b | \bEndosome\b | \bExtracellular\b | \bGolgi apparatus\b | \bLysosome\b | \bMitochondrion\b | \bNucleus\b | \bPeroxisome\b | \bPlasma membrane', flags=re.I | re.X)
#     line_number = 0
#     out_line_number = 1
#     humploc_id = None
#     humploc_ac = None
#     humploc_loc = None
#     for line in f:
#         if re.search("//", line):
#             line_number = 0
#         else:
#             line_number += 1
#         if line_number == 1:
#             humploc_ac = line
#         humploc_id = re.search('(.+)_HUMAN', line)
#         humploc_loc = humploc_loc_re.findall(line)
#         if humploc_id and humploc_loc and humploc_ac:
#             out_line_number += 1
#             
# # probably unnecessary block
# 
# cols = None
# unis = None
# hpas = None 
# 
# unis = list(uniprot.columns)
# unis = ["UNIPROT_" + s for s in unis]
# 
# hpas = list(hpa.columns)
# hpas = ["HPA_" +  s for s in hpas]
# 
# cols = [unis, hpas]
# 
# cols = [item for sublist in cols for item in sublist]
# cols = ['HUMPLOC_AC', 'HUMPLOC_ID', 'HUMPLOC_LOC'] + cols # this is probably wrong, and the whole block should be redone
# 
# out = pd.DataFrame(columns=cols)
# out